# Ice Sheet Emulator Demo

### Installation

To install, run this is your terminal. This will create an environment, download ISE, and activate a jupyter notebook instance. Open up this notebook to run the model.  

```conda create -n ise -y```  
```conda activate ise```  
```conda install nb_conda ipykernel -y```  
```pip install git+https://github.com/Brown-SciML/ise```  
```jupyter notebook```  

In [1]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from ise.models.hybrid import HybridEmulator, DeepEnsemble, NormalizingFlow

### Synthetic Data and Scaling

In [2]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]], dtype=int)
y = np.array([[1], [2], [3],], dtype=int)

scaler_X = StandardScaler().fit(X)
scaler_y = StandardScaler().fit(y)
X = scaler_X.transform(X)
y = scaler_y.transform(y)

### Create emulator Model
Create the model with the Deep Ensemble as the Predictor and the Normalizing Flow as the uncertainty quantifier.

In [3]:
input_shape = X.shape[1]
output_shape = y.shape[1]
num_ensemble_members = 2

predictor = DeepEnsemble(num_predictors=num_ensemble_members, forcing_size=input_shape, sle_size=output_shape)
uncertainty_quantifier = NormalizingFlow(forcing_size=input_shape, sle_size=output_shape)
emulator = HybridEmulator(deep_ensemble=predictor, normalizing_flow=uncertainty_quantifier)

In [4]:
emulator.fit(X, y, nf_epochs=10, de_epochs=15)


Training Normalizing Flow (10 epochs):
Epoch 1, Loss: 8.679369926452637
Epoch 2, Loss: 8.459479331970215
Epoch 3, Loss: 8.245292663574219
Epoch 4, Loss: 8.0366849899292
Epoch 5, Loss: 7.833608627319336
Epoch 6, Loss: 7.636020183563232
Epoch 7, Loss: 7.443871974945068
Epoch 8, Loss: 7.257107257843018


C:\Users\pvank\miniconda3\envs\ise\Lib\site-packages\ise\data\dataclasses.py:33: UserWarning: Full projections of 86 timesteps are not present in the dataset. This may lead to unexpected behavior.
  warnings.warn(


Epoch 9, Loss: 7.075656414031982
Epoch 10, Loss: 6.899447917938232

Training Deep Ensemble (15 epochs):
Training Weak Predictor 1 of 2:
Epoch 1, Average Batch Loss: 0.48592087626457214
Epoch 2, Average Batch Loss: 0.48054543137550354
Epoch 3, Average Batch Loss: 0.47530093789100647
Epoch 4, Average Batch Loss: 0.46996942162513733
Epoch 5, Average Batch Loss: 0.46499523520469666
Epoch 6, Average Batch Loss: 0.45987486839294434
Epoch 7, Average Batch Loss: 0.45409807562828064
Epoch 8, Average Batch Loss: 0.4478793144226074
Epoch 9, Average Batch Loss: 0.4412558078765869
Epoch 10, Average Batch Loss: 0.43391919136047363
Epoch 11, Average Batch Loss: 0.4259643256664276
Epoch 12, Average Batch Loss: 0.4172305166721344
Epoch 13, Average Batch Loss: 0.4077059328556061
Epoch 14, Average Batch Loss: 0.39677444100379944
Epoch 15, Average Batch Loss: 0.3842660188674927

Training Weak Predictor 2 of 2:
Epoch 1, Average Batch Loss: 0.8563575744628906
Epoch 2, Average Batch Loss: 0.8397316932678223


### Predict on unseen data

In [5]:
predictions, uncertainties = emulator.predict(scaler_X.transform(np.array([[10, 11, 12]])), output_scaler=scaler_y)

C:\Users\pvank\miniconda3\envs\ise\Lib\site-packages\ise\data\dataclasses.py:33: UserWarning: Full projections of 86 timesteps are not present in the dataset. This may lead to unexpected behavior.
  warnings.warn(


In [6]:
predictions

array([[2.176648]], dtype=float32)

In [7]:
uncertainties

{'total': array([[6.5961537]], dtype=float32),
 'epistemic': array([[0.00804853]], dtype=float32),
 'aleatoric': array([[6.588105]], dtype=float32)}

### Save emulator and delete emulator variable

In [8]:
if not os.path.exists('./saved_model/'):
    os.mkdir('./saved_model/')
emulator.save(save_dir='./saved_model/')

Model metadata saved to ./saved_model//deep_ensemble_metadata.json
Model parameters saved to ./saved_model//deep_ensemble.pth
WeakPredictor 1 model parameters saved to ./saved_model\weak_predictors\weak_predictor_1.pth
WeakPredictor 2 model parameters saved to ./saved_model\weak_predictors\weak_predictor_2.pth
Model and metadata saved to ./saved_model//normalizing_flow.pth and ./saved_model//normalizing_flow.pth_metadata.json, respectively.


In [9]:
del emulator

### Calculate total size of Emulator

In [10]:
import os

def get_total_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return round(total_size / 1048576, 2)

print(f"Total size of saved Emulator in '{'./saved_model/'}': {get_total_directory_size('./saved_model/')} MB")


Total size of saved Emulator in './saved_model/': 13.28 MB


### Load model and predict to ensure parameters loaded correctly

In [11]:
emulator = HybridEmulator.load('./saved_model/')

In [12]:
predictions, uncertainties = emulator.predict(scaler_X.transform(np.array([[10, 11, 12]])), output_scaler=scaler_y)

C:\Users\pvank\miniconda3\envs\ise\Lib\site-packages\ise\data\dataclasses.py:33: UserWarning: Full projections of 86 timesteps are not present in the dataset. This may lead to unexpected behavior.
  warnings.warn(


In [13]:
predictions

array([[2.176648]], dtype=float32)

In [14]:
uncertainties

{'total': array([[7.313538]], dtype=float32),
 'epistemic': array([[0.00804853]], dtype=float32),
 'aleatoric': array([[7.3054895]], dtype=float32)}